In [1]:
import pandas as pd
import numpy as np
from datetime import datetime
import pickle
from fuzzywuzzy import fuzz, process
from difflib import SequenceMatcher
pd.set_option('display.max_columns', 100)

In [2]:
today = f"{datetime.now():%Y%m%d}"
open_file = open(f'20220917_datadict.pkl', "rb")
datadict = pickle.load(open_file)
open_file.close()

In [3]:
open_file = open(f'scores.pkl', "rb")
scores = pickle.load(open_file)
open_file.close()

In [4]:
def similar(a, b):
    return SequenceMatcher(None, a, b).ratio()

In [5]:
def getname(name, l):
    if len(l) == 0:
        return ''
    highest = process.extractOne(name,l)
    return highest[0]

In [86]:
datadict

[{'Expected goals (xG)': ['0.33', '0.10'],
  'Total shots': ['4', '1'],
  'Big chances': ['1', '0'],
  'Big chances missed': ['1', '0'],
  'Accurate passes': ['123', '116'],
  'Accurate passes percentage': [0.83, 0.78],
  'Fouls committed': ['5', '3'],
  'Offsides': ['0', '0'],
  'Corners': ['0', '2'],
  'Shots off target': ['2', '1'],
  'Shots on target': ['2', '0'],
  'Blocked shots': ['0', '0'],
  'Hit woodwork': ['0', '0'],
  'Shots inside box': ['1', '1'],
  'Shots outside box': ['3', '0'],
  'xG open play': ['0.27', '0.12'],
  'xG set play': ['0.05', '0.10'],
  'xG on target (xGOT)': ['0.92', '0.00'],
  'Passes': ['149', '149'],
  'Own half': ['70', '84'],
  'Opposition half': ['53', '32'],
  'Accurate long balls': ['15', '3'],
  'Accurate long balls percentage': [0.6, 0.16],
  'Accurate crosses': ['3', '1'],
  'Accurate crosses percentage': [0.43, 0.2],
  'Throws': ['9', '9'],
  'Yellow cards': ['2', '2'],
  'Red cards': ['0', '0'],
  'Tackles won': ['6', '4'],
  'Tackles won pe

In [25]:
res = []
for dic in datadict:
    e_dict = {}
    for name, tup in dic.items():
        name = name.replace(' ','_').lower()
        if len(tup) == 2 and isinstance(tup, list) and (isinstance(tup[0], str) or 
                                                       isinstance(tup[0], float)):
            e_dict[f'{name}_diff'] = float(tup[0]) - float(tup[1])
#             e_dict[f'{name}_home'] = float(tup[0])
#             e_dict[f'{name}_away'] = float(tup[1])
#             e_dict[f'{name}_sum'] = float(tup[0]) + float(tup[1])
        else:
            e_dict[name] = tup
    if 'bet' in e_dict:
        e_dict['odds_sum'] = e_dict['bet']['odds_checker_sum']
        del e_dict['bet']
    e_dict['median_odds'] = e_dict.pop('odds')
    if len(e_dict['median_odds']) > 0:
        e_dict['std_odds'] = dict(zip(list(e_dict['median_odds'].keys()),
                                      [np.std(sorted(x)[2:-2]) for x in e_dict['full_odds']]))
        e_dict['75_odds'] = dict(zip(list(e_dict['median_odds'].keys()),
                                      [np.percentile(sorted(x), 75) for x in e_dict['full_odds']]))
    res.append(e_dict)

C:\Users\idotz\anaconda3\lib\site-packages\numpy\core\_methods.py:265: RuntimeWarning: Degrees of freedom <= 0 for slice
  ret = _var(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
C:\Users\idotz\anaconda3\lib\site-packages\numpy\core\_methods.py:223: RuntimeWarning: invalid value encountered in divide
  arrmean = um.true_divide(arrmean, div, out=arrmean, casting='unsafe',
C:\Users\idotz\anaconda3\lib\site-packages\numpy\core\_methods.py:257: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


In [26]:
df = pd.DataFrame(res)
df = df[df['half_time'] == 'Half-Time']
df = df[(df['median_odds'] != '') & (df['median_odds'] != {})]
df = df.reset_index(drop = True)
df['o_names'] = df['odds_url'].apply(lambda x: ['draw'] + x.split('/')[-2].split('-v-'))
for name in ['std','median','75']:
    r = []
    for i in df[[f'{name}_odds','o_names']].to_numpy():
        match = [getname(name, list(i[0].keys())) for name in i[1]]
        match = [i[0][key] for key in match]
        r.append(match)
    df[f'{name}_odds'] = r
df['odds_predict'] = df['median_odds'].apply(lambda x: np.argmin(x))
ordo_df = pd.DataFrame(df['median_odds'].tolist(), columns=['med_0', 'med_1', 'med_2'])
std_o = pd.DataFrame(df['std_odds'].tolist(), columns=['std_0', 'std_1', 'std_2'])
sevfiv_o = pd.DataFrame(df['75_odds'].tolist(), columns=['75_0', '75_1', '75_2'])
df = pd.concat([df, std_o, ordo_df,sevfiv_o], axis=1)
df[['med_odds_prob_0','med_odds_prob_1','med_odds_prob_2']] = df[['med_0','med_1','med_2']].apply(lambda x: 1/x)
df[['75_odds_prob_0','75_odds_prob_1','75_odds_prob_2']] = df[['75_0','75_1','75_2']].apply(lambda x: 1/x)
df = df.drop([ 'half_time','o_names', '75_odds', 'median_odds',
             'std_odds', 'full_odds','match_name'], axis = 1)

In [27]:
df

,expected_goals_(xg)_diff,total_shots_diff,big_chances_diff,big_chances_missed_diff,accurate_passes_diff,accurate_passes_percentage_diff,fouls_committed_diff,offsides_diff,corners_diff,shots_off_target_diff,shots_on_target_diff,blocked_shots_diff,hit_woodwork_diff,shots_inside_box_diff,shots_outside_box_diff,xg_open_play_diff,xg_set_play_diff,xg_on_target_(xgot)_diff,passes_diff,own_half_diff,opposition_half_diff,accurate_long_balls_diff,accurate_long_balls_percentage_diff,accurate_crosses_diff,accurate_crosses_percentage_diff,throws_diff,yellow_cards_diff,red_cards_diff,tackles_won_diff,tackles_won_percentage_diff,interceptions_diff,blocks_diff,clearances_diff,keeper_saves_diff,duels_won_diff,ground_duels_won_diff,ground_duels_won_percentage_diff,aerial_duels_won_diff,aerial_duels_won_percentage_diff,successful_dribbles_diff,successful_dribbles_percentage_diff,score_diff,posession_diff,url,odds_url,odds_sum,xg_penalty_diff,odds_predict,std_0,std_1,std_2,med_0,med_1,med_2,75_0,75_1,75_2,med_odds_prob_0,med_odds_prob_1,med_odds_prob_2,75_odds_prob_0,75_odds_prob_1,75_odds_prob_2
0,0.89,7.0,1.0,0.0,77.0,0.06,-3.0,1.0,1.0,4.0,0.0,3.0,0.0,7.0,0.0,0.97,-0.08,0.28,77.0,43.0,34.0,8.0,0.13,3.0,0.23,2.0,-2.0,0.0,-3.0,0.26,2.0,-3.0,-1.0,0.0,-2.0,-1.0,-0.03,-1.0,-0.10,4.0,-0.31,0.0,20.0,https://www.fotmob.com/match/3888550/matchfact...,https://www.oddschecker.com/football/world/mex...,1.091726,NaN,0,0.057012,0.039078,0.166599,2.300,2.387500,4.200000,2.393750,2.400000,4.266667,0.434783,0.418848,0.238095,0.417755,0.416667,0.234375
1,-1.21,-5.0,-1.0,0.0,-54.0,-0.09,9.0,-1.0,-1.0,-1.0,-2.0,-2.0,0.0,-4.0,-1.0,-0.96,-0.26,-1.28,-49.0,5.0,-59.0,-7.0,-0.15,-1.0,0.00,-3.0,2.0,0.0,5.0,0.40,0.0,2.0,9.0,0.0,-3.0,-5.0,-0.16,2.0,0.06,-2.0,-0.11,-1.0,-16.0,https://www.fotmob.com/match/3888551/matchfact...,https://www.oddschecker.com/football/world/mex...,1.087449,NaN,2,0.191021,0.863459,0.016510,4.750,13.000000,1.250000,5.000000,14.000000,1.250000,0.210526,0.076923,0.800000,0.200000,0.071429,0.800000
2,-0.74,-3.0,-1.0,0.0,-140.0,-0.11,-3.0,0.0,-1.0,0.0,-1.0,-2.0,0.0,-1.0,-2.0,-0.06,-0.67,-1.25,-141.0,-31.0,-109.0,-7.0,-0.30,-1.0,-0.01,-5.0,-2.0,0.0,-2.0,0.43,4.0,1.0,5.0,-1.0,-15.0,-8.0,-0.40,-7.0,-0.64,-4.0,-0.80,-2.0,-26.0,https://www.fotmob.com/match/3900373/matchfact...,https://www.oddschecker.com/football/netherlan...,1.044048,NaN,2,2.616801,18.683692,0.004381,20.000,56.000000,1.024390,21.000000,81.000000,1.025000,0.050000,0.017857,0.976190,0.047619,0.012346,0.975610
3,0.37,4.0,1.0,1.0,56.0,0.04,-1.0,-1.0,-1.0,3.0,2.0,-1.0,1.0,2.0,2.0,0.37,NaN,0.97,56.0,-14.0,70.0,-4.0,-0.01,4.0,0.40,-2.0,-2.0,0.0,-3.0,-0.17,-4.0,1.0,-6.0,-3.0,-5.0,1.0,0.02,-6.0,-0.36,3.0,0.04,-1.0,12.0,https://www.fotmob.com/match/3904422/matchfact...,https://www.oddschecker.com/football/france/li...,1.064560,NaN,2,0.066525,0.227323,0.016281,3.500,6.500000,1.600000,3.550000,6.500000,1.615385,0.285714,0.153846,0.625000,0.281690,0.153846,0.619048
4,-2.94,-12.0,-3.0,0.0,-242.0,-0.33,-1.0,-1.0,-4.0,0.0,-6.0,-6.0,1.0,-10.0,-2.0,-2.18,-0.76,-3.13,-234.0,-105.0,-137.0,-4.0,-0.31,-1.0,0.13,-8.0,1.0,0.0,1.0,0.07,3.0,6.0,13.0,2.0,7.0,5.0,0.24,2.0,0.10,3.0,0.60,-4.0,-50.0,https://www.fotmob.com/match/3902828/matchfact...,https://www.oddschecker.com/football/scottish/...,0.970938,NaN,2,244.528596,270.608007,0.028740,226.000,201.000000,1.040000,538.500000,501.000000,1.070455,0.004425,0.004975,0.961538,0.001857,0.001996,0.934183
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
517,0.74,2.0,1.0,1.0,64.0,-0.01,5.0,0.0,0.0,2.0,0.0,0.0,0.0,1.0,1.0,0.74,NaN,0.14,75.0,-17.0,81.0,4.0,-0.05,-1.0,-0.33,-4.0,0.0,0.0,2.0,0.17,-9.0,0.0,-1.0,-1.0,-9.0,-5.0,-0.22,-4.0,-0.50,-2.0,-0.60,-1.0,12.0,https://www.fotmob.com/match/3916999/matchfact...,https://www.oddschecker.com/football/belgium/j...,1.085579,NaN

In [28]:
df['target'] = scores

In [29]:
df['url'] = df['url'].apply(lambda x: x.split('/')[-1])
df['odds_url'] = df['odds_url'].apply(lambda x: x.split('/')[-2])

In [30]:
test = df[['url','odds_url']]
test['s'] = test[['url','odds_url']].apply(lambda x: similar(x.url,x.odds_url), axis = 1)
test = test.sort_values(by = 's')

C:\Users\idotz\anaconda3\lib\site-packages\pandas\core\frame.py:3607: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._set_item(key, value)


In [31]:
remove = [42,254,18,326,22,319,76, 514]

In [32]:
df = df.drop(remove)
df = df.drop(['url','odds_url'], axis = 1)

In [33]:
df = df.reset_index(drop = True)
df = df[df['odds_sum'] > 1.02]

In [115]:
df

,expected_goals_(xg),total_shots,big_chances,big_chances_missed,accurate_passes,accurate_passes_percentage,fouls_committed,offsides,corners,shots_off_target,shots_on_target,blocked_shots,hit_woodwork,shots_inside_box,shots_outside_box,xg_open_play,xg_set_play,xg_on_target_(xgot),passes,own_half,opposition_half,accurate_long_balls,accurate_long_balls_percentage,accurate_crosses,accurate_crosses_percentage,throws,yellow_cards,red_cards,tackles_won,tackles_won_percentage,interceptions,blocks,clearances,keeper_saves,duels_won,ground_duels_won,ground_duels_won_percentage,aerial_duels_won,aerial_duels_won_percentage,successful_dribbles,successful_dribbles_percentage,score,posession,odds_sum,xg_penalty,odds_predict,std_0,std_1,std_2,med_0,med_1,med_2,75_0,75_1,75_2,med_odds_prob_0,med_odds_prob_1,med_odds_prob_2,75_odds_prob_0,75_odds_prob_1,75_odds_prob_2,target
0,"[1.00, 0.11]","[8, 1]","[1, 0]","[0, 0]","[193, 116]","[0.87, 0.81]","[5, 8]","[1, 0]","[3, 2]","[4, 0]","[1, 1]","[3, 0]","[0, 0]","[8, 1]","[0, 0]","[0.97, 0.00]","[0.03, 0.11]","[0.39, 0.11]","[221, 144]","[113, 70]","[80, 46]","[16, 8]","[0.55, 0.42]","[4, 1]","[0.33, 0.1]","[8, 6]","[0, 2]","[0, 0]","[5, 8]","[0.83, 0.57]","[5, 3]","[0, 3]","[9, 10]","[0, 0]","[26, 28]","[21, 22]","[0.49, 0.52]","[5, 6]","[0.45, 0.55]","[7, 3]","[0.44, 0.75]","[1, 1]","[60, 40]",1.091726,NaN,0,0.075452,0.045693,0.212633,2.300000,2.387500,4.200000,2.393750,2.400000,4.266667,0.434783,0.418848,0.238095,0.417755,0.416667,0.234375,0
1,"[0.13, 1.34]","[5, 10]","[0, 1]","[0, 0]","[104, 158]","[0.75, 0.84]","[14, 5]","[0, 1]","[3, 4]","[3, 4]","[1, 3]","[1, 3]","[0, 0]","[2, 6]","[3, 4]","[0.10, 1.06]","[0.03, 0.29]","[0.03, 1.31]","[139, 188]","[67, 62]","[37, 96]","[10, 17]","[0.5, 0.65]","[1, 2]","[0.17, 0.17]","[4, 7]","[3, 1]","[0, 0]","[6, 1]","[0.6, 0.2]","[4, 4]","[3, 1]","[14, 5]","[1, 1]","[27, 30]","[18, 23]","[0.43, 0.59]","[9, 7]","[0.5, 0.44]","[3, 5]","[0.6, 0.71]","[1, 2]","[42, 58]",1.087449,NaN,2,0.286912,1.108553,0.021276,4.750000,13.000000,1.250000,5.000000,14.000000,1.250000,0.210526,0.076923,0.800000,0.200000,0.071429,0.800000,2
2,"[0.17, 0.91]","[3, 6]","[0, 1]","[0, 0]","[141, 281]","[0.75, 0.86]","[3, 6]","[0, 0]","[1, 2]","[2, 2]","[1, 2]","[0, 2]","[0, 0]","[2, 3]","[1, 3]","[0.17, 0.23]","[0.00, 0.67]","[0.12, 1.37]","[187, 328]","[83, 114]","[58, 167]","[8, 15]","[0.33, 0.63]","[3, 4]","[0.3, 0.31]","[9, 14]","[0, 2]","[0, 0]","[2, 4]","[1.0, 0.57]","[7, 3]","[1, 0]","[13, 8]","[0, 1]","[8, 23]","[6, 14]","[0.3, 0.7]","[2, 9]","[0.18, 0.82]","[0, 4]","[0.0, 0.8]","[0, 2]","[37, 63]",1.044048,NaN,2,3.103652,20.120048,0.005754,20.000000,56.000000,1.024390,21.000000,81.000000,1.025000,0.050000,0.017857,0.976190,0.047619,0.012346,0.975610,2
3,"[0.43, 0.06]","[6, 2]","[1, 0]","[1, 0]","[218, 162]","[0.84, 0.8]","[4, 5]","[1, 2]","[1, 2]","[3, 0]","[3, 1]","[0, 1]","[1, 0]","[4, 2]","[2, 0]","[0.43, 0.06]",NaN,"[1.08, 0.11]","[259, 203]","[94, 108]","[124, 54]","[14, 18]","[0.48, 0.49]","[4, 0]","[0.4, 0.0]","[8, 10]","[0, 2]","[0, 0]","[1, 4]","[0.5, 0.67]","[4, 8]","[1, 0]","[7, 13]","[0, 3]","[17, 22]","[12, 11]","[0.52, 0.5]","[5, 11]","[0.33, 0.69]","[5, 2]","[0.71, 0.67]","[0, 1]","[56, 44]",1.064560,NaN,2,0.093617,0.235317,0.018924,3.500000,6.500000,1.600000,3.550000,6.500000,1.615385,0.285714,0.153846,0.625000,0.281690,0.153846,0.619048,1
5,"[0.31, 1.26]","[5, 4]","[0, 2]","[0, 0]","[166, 187]","[0.81, 0.83]","[5, 5]","[1, 0]","[3, 1]","[2, 0]","[2, 3]","[1, 1]","[0, 0]","[4, 4]","[1, 0]","[0.12, 0.16]","[0.19, 0.32]","[0.09, 2.08]","[206, 225]","[62, 121]","[104, 66]","[14, 15]","[0.67, 0.47]","[2, 2]","[0.29, 0.33]","[12, 7]","[1, 1]","[0, 0]","[3, 7]","[0.6, 0.88]","[10, 6]","[1, 1]","[2, 6]","[1, 1]","[16, 21]","[10, 16]","[0.38, 0.64]","[6, 5]","[0.55, 0.5]","[3, 3]","[0.5, 0.75]","[1, 2]","[48, 52]",1.092484,"[0.00, 0.78]",2,0.131783,0.154580,0.049411,3.126923,4.500000,1.816667,3.237500,4.712500,1.833333,0.319803,0.222222,0.550459,0.308880,0.212202,0.5454

In [34]:
df.to_csv('f_eng_df.csv')